# 벚꽃 개화시기를 예측해봅시다


# Before We Begin...

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic')

The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 148489 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly insta

In [ ]:
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen, urlretrieve
from urllib.parse import urlencode, quote_plus, unquote, urlparse
import os
from datetime import datetime
import statistics
from scipy.stats.stats import pearsonr
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn import datasets, linear_model

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive', force_remount=True)          

Mounted at /content/drive


# Loading Data for Model Training

## Read Flowering CSV files

In [ ]:
path = '/content/drive/MyDrive/DATA/'
headers = ['code', 'lat', 'lon', 'year', 'date']

df_g = pd.read_csv(path + 'season_g.csv', header=None, names=headers, index_col=None)
df_f = pd.read_csv(path + 'season_f.csv', header=None, names=headers, index_col=None)
df_fb = pd.read_csv(path + 'season_fb.csv', header=None, names=headers, index_col=None)

## Read Station Values

In [ ]:
germ_codes = df_g['code'].values.tolist()
flow_codes = df_f['code'].values.tolist()
fullb_codes = df_fb['code'].values.tolist()

def get_codes(codes):
    temp_codes = []
    for i in range(len(codes)):
        if i == 0:
            temp_codes.append(codes[i])
        else:
            if codes[i] != temp_codes[-1]:
                temp_codes.append(codes[i])
    return temp_codes

# Unique Observatory Codes for each dataset
u_germ_codes = get_codes(germ_codes)
u_flow_codes = get_codes(flow_codes)
u_fullb_codes = get_codes(fullb_codes)

In [ ]:
df_stations = pd.read_csv(path + 'ASOS_stations.csv', header=0, index_col=0)

In [ ]:
# Save data of observation station gps coordinates

gps_array = np.zeros((len(u_flow_codes), 3))
gps_array[:,0] = u_flow_codes

for i, code in enumerate(u_flow_codes):
    for x, row in df_f.iterrows():
        if row['code'] == code:
            gps_array[i, 1] = row['lat']
            gps_array[i, 2] = row['lon']
            break

## Read Temperature CSV Files

In [ ]:
def read_csv_temp_year(stnId, year):
    filepath = '/{}/'.format(stnId)
    filename = '{}_{}.csv'.format(year, stnId)
    df = pd.read_csv(path + filepath + filename, sep=',', header=0, index_col=None, usecols=[1,2,3,4])
    return df

def read_csv_temp(stnId):
    np_result = np.zeros((1, 4))
    filepath = '/{}/'.format(stnId)
    foldername = '{}'.format(stnId)
    if foldername in os.listdir(path):
        for i in np.arange(1961, 2023, 2):
            filename = '{}_{}.csv'.format(i, stnId)
            if filename in os.listdir(path+filepath):
                df_temp = read_csv_temp_year(stnId, i)
                #print(df_temp.head())
                np_temp = df_temp.to_numpy()
                np_result = np.vstack((np_result, np_temp))
    np_result = np.delete(np_result, (0), axis=0)
    df = pd.DataFrame(np_result, columns=['date', 'avgT', 'minT', 'maxT'])
    df = df.dropna(subset=['avgT', 'minT', 'maxT'])
    return df

In [ ]:
# Get all data from all stations
def read_date(datestring):
    temp_string = datestring.split('-')
    if len(temp_string) == 3:
        for i, item in enumerate(temp_string):
            temp_string[i] = int(item)
    else:
        temp_string = datestring.split('/')
        reverse_temp_string = temp_string
        for i, item in enumerate(temp_string):
            if i < 2:
                reverse_temp_string[i+1] = int(item)
            else:
                reverse_temp_string[-1] = int(item)
        temp_string = reverse_temp_string
    return temp_string

def preprocess_temp_date(temparray):
    resultarray = temparray
    date_array = np.empty((len(temparray[:,0]), 4))
    for i, date in enumerate(temparray[:, 0]):
        temp_date = read_date(date)
        dt = datetime(int(temp_date[0]), int(temp_date[1]), int(temp_date[2]))
        jd = '%03d' % (dt.timetuple().tm_yday)
        jd  = int(jd)
        temp_date.append(jd)
        date_array[i, :] = temp_date
    resultarray = np.delete(resultarray, 0, axis=1)
    resultarray = np.hstack((date_array[:, 3].reshape(-1, 1), resultarray))
    resultarray = np.hstack((date_array[:, 2].reshape(-1, 1), resultarray))
    resultarray = np.hstack((date_array[:, 1].reshape(-1, 1), resultarray))
    resultarray = np.hstack((date_array[:, 0].reshape(-1, 1), resultarray))
    return resultarray

data_dict = {}
stations_not_counted = []
print('Reading Temperature Data')
for i, rows in df_stations.iterrows():
    df_temp = read_csv_temp(stnId=i)
    if not df_temp.empty:
        temp_array = df_temp.to_numpy()
        print('station : {}'.format(i))
        data_dict['{}'.format(i)] = preprocess_temp_date(temp_array)
    else:
        stations_not_counted.append(i) # removing empty stations

# data_dict
# year  month   day     jday    avgT    minT    maxT

## Read Seoul (108) Station Data

## Processing Data for Easy Manipulation

In [ ]:
def preprocess_graph(codes, df):
    temp_array = np.zeros((len(range(1920,2022)), len(codes)+1))
    temp_array[:,0] = range(1920,2022)

    for i, code in enumerate(codes):
        for x, row in df.iterrows():
            if row['code'] == code:
                temp_array[int(row['year'])-1920, i+1] = row['date']

    for i in range(len(temp_array[0,:])):
        for j in range(len(temp_array[:,0])):
            if temp_array[j, i] <= 10:
                temp_array[j,i] = np.nan
    return temp_array

germ_array = preprocess_graph(u_germ_codes, df_g)
flow_array = preprocess_graph(u_flow_codes, df_f)
fullb_array = preprocess_graph(u_fullb_codes, df_fb)

### Differences Between Dates Graph

In [ ]:
flow_germ = []
fullb_germ = []
fullb_flow = []

for i, date in enumerate(seoul_germ):
    if date >= 1:
        if seoul_flow[i] >= 1:
            flow_germ.append(seoul_flow[i] - date)
        else:
            flow_germ.append(np.nan)
    else:
        flow_germ.append(np.nan)
for i, date in enumerate(seoul_germ):
    if date >= 1:
        if seoul_fullb[i] >= 1:
            fullb_germ.append(seoul_fullb[i] - date)
        else:
            fullb_germ.append(np.nan)
    else:
        fullb_germ.append(np.nan)
for i, date in enumerate(seoul_flow):
    if date >= 1:
        if seoul_fullb[i] >= 1:
            fullb_flow.append(seoul_fullb[i] - date)
        else:
            fullb_flow.append(np.nan)
    else:
        fullb_flow.append(np.nan)

fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(10,12), sharex=False, sharey=True)
fig.suptitle('Differences Between Dates')

axs[0].plot(germ_array[:,0], flow_germ, color='black')
sum = 0
for i in flow_germ:
    if i > 0:
        sum += i
avg0 = sum / len(flow_germ)
axs[0].plot(germ_array[:,0], [avg0] * len(flow_germ), color='black', linestyle='dashed')
axs[1].plot(germ_array[:,0], fullb_germ, color='black')
sum = 0
for i in fullb_germ:
    if i > 0:
        sum += i
avg1 = sum / len(fullb_germ)
axs[1].plot(germ_array[:,0], [avg1] * len(flow_germ), color='black', linestyle='dashed')
axs[2].plot(germ_array[:,0], fullb_flow, color='black')
sum = 0
for i in fullb_flow:
    if i > 0:
        sum += i
avg2 = sum / len(fullb_flow)
axs[2].plot(germ_array[:,0], [avg2] * len(flow_germ), color='black', linestyle='dashed')

axs[0].title.set_text('Flowering - Germination, avg={0:.3f}, std={1:.3f}'.format(avg0, np.nanstd(flow_germ)))
axs[0].grid()
axs[1].title.set_text('Full Bloom - Germination, avg={0:.3f}, std={1:.3f}'.format(avg1, np.nanstd(fullb_germ)))
axs[1].grid()
axs[2].title.set_text('Full Bloom - Flowering, avg={0:.3f}, std={1:.3f}'.format(avg2, np.nanstd(fullb_flow)))
axs[2].grid()

fig.text(0.5, 0.1, 'Year', ha = 'center')
fig.text(0.05, 0.5, 'Date', va='center', rotation='vertical')
plt.show()

In [ ]:
df_stations = df_stations.drop(stations_not_counted)

df_stations['lat'] = gps_array[:,1]
df_stations['lon'] = gps_array[:,2]

df_stations.to_csv(path + 'stations_info.csv')

# Calculation of GDD (Growing Degree-Days) Index

In [ ]:
def calculate_gdd_year(year, stnId, endday, critical_temp):
    if endday > 0:
        temp_data = data_dict[str(stnId)][:,[0,3,4]]
        gdd = 0
        for i in range(len(temp_data[:,0])):
            if temp_data[i,0] == year:
                if temp_data[i, 1] < endday:
                    gdd += max(temp_data[i,2] - critical_temp, 0)
    else:
        gdd = np.nan
    return gdd

def calculate_chill_day(year, stnId, endday, critical_temp):
    if endday > 0:
        temp_data = data_dict[str(stnId)][:,[0,3,4]]
        count = 0
        year_change = 0
        for i in range(len(temp_data[:,0])):
            if temp_data[i,0] == year - 1:
                if temp_data[i,1] >= 300:
                    year_change = 1
                    if temp_data[i,2] <= critical_temp:
                        count += 1
            elif temp_data[i,0] == year and year_change == 1:
                if temp_data[i,1] < endday:
                    if temp_data[i,2] <= critical_temp:
                        count += 1
    else:
        count = np.nan
    return count

def calculate_chill_gdd(year, stnId, endday, critical_temp):
    if endday >= 0:
        temp_data = data_dict[str(stnId)][:,[0,3,4]]
        gdd = 0
        year_change = 0
        for i in range(len(temp_data[:,0])):
            if temp_data[i,0] == year - 1:
                if temp_data[i,1] >= 300:
                    year_change = 1
                    gdd += min(0, temp_data[i,2] - critical_temp)
            elif temp_data[i,0] == year and year_change == 1:
                if temp_data[i,1] < endday:
                    gdd += min(0, temp_data[i,2] - critical_temp)
    else:
        gdd = np.nan
    return gdd
            

def get_day(ftype, stnId, year):
    if ftype == 'g':
        for i, y in enumerate(germ_array[:,0]):
            if y == year:
                for j, code in enumerate(u_germ_codes):
                    if int(stnId) == code:
                        day = germ_array[i, j+1]
                        return day
    elif ftype == 'f':
        for i, y in enumerate(flow_array[:,0]):
            if y == year:
                for j, code in enumerate(u_flow_codes):
                    if int(stnId) == code:
                        day = flow_array[i, j+1]
                        return day
    elif ftype == 'fb':
        for i, y in enumerate(fullb_array[:,0]):
            if y == year:
                for j, code in enumerate(u_fullb_codes):
                    if int(stnId) == code:
                        day = fullb_array[i, j+1]
                        return day
    else:
        print('invalid type code; try g, f, fb')

def calculate_gdd(ftype, stnId, critical_temp):
    temp_data = data_dict[str(stnId)][:,0]
    temp_start_year = temp_data[0]
    temp_end_year = temp_data[-1]
    if ftype == 'g':
        date_data = germ_array[:,0]
    elif ftype == 'f':
        date_data = flow_array[:,0]
    elif ftype == 'fb':
        date_data = fullb_array[:,0]
    else:
        print('invalid type code; try g, f, fb')
    date_start_year = date_data[0]
    date_end_year = date_data[-1]
    start_year = int(max(temp_start_year, date_start_year))
    end_year = int(min(temp_end_year, date_end_year))
    gdd = []
    dates = []
    stn = []
    years = range(start_year, end_year + 1)
    for year in years:
        day = get_day(ftype, stnId, year)
        if day != 0:
            gdd.append(calculate_gdd_year(year, stnId, day, critical_temp))
            dates.append(day)
            stn.append(stnId)
        else:
            gdd.append(np.nan)
            dates.append(np.nan)
            stn.append(stnId)
    gdd = np.array(gdd).reshape(-1,1)
    dates = np.array(dates).reshape(-1,1)
    years = np.array(years).reshape(-1,1)
    stn = np.array(stn).reshape(-1,1)
    return np.hstack((stn, years, dates, gdd))
    # stationID, year, budding date, GDD sum

def get_criticaldates(ftype, stnId):
    critical_temp = 5

    temp_data = data_dict[str(stnId)][:,0]
    temp_start_year = temp_data[0]
    temp_end_year = temp_data[-1]
    if ftype == 'g':
        date_data = germ_array[:,0]
    elif ftype == 'f':
        date_data = flow_array[:,0]
    elif ftype == 'fb':
        date_data = fullb_array[:,0]
    else:
        print('invalid type code; try g, f, fb')
    date_start_year = date_data[0]
    date_end_year = date_data[-1]
    start_year = int(max(temp_start_year, date_start_year))
    end_year = int(min(temp_end_year, date_end_year))
    years = range(start_year, end_year + 1)

    critical_dates = []
    temp_gdd_array = calculate_gdd(ftype, stnId, critical_temp)
    avg = 0
    for i in temp_gdd_array[:,3]:
        if i > 0:
            avg += i
    avg = avg / len(temp_gdd_array[:,3])
    for year in years:
        for j in range(150):
            temp = calculate_gdd_year(year, 108, j, critical_temp)
            if temp >= avg:
                critical_dates.append(j)
                break
    np_crit = np.array(critical_dates).reshape(-1,1)
    return np.hstack((temp_gdd_array, np_crit))
    # stationID, year, budding date, (heating) GDD sum, (heating) critical date

In [ ]:
# seoul_germ_array = stnID, year, budding date, (heating) GDD sum_budding, (heating) critical date_budding, flowering date

seoul_germ_array = get_criticaldates('g', 108)
seoul_flow_array = get_criticaldates('f', 108)

ffd_dates = []
for year in range(1961, 2021):
    day = get_day('f', 108, year)
    if day != 0:
        ffd_dates.append(day)
    else:
        ffd_dates.append(np.nan)

np_ffd = np.array(ffd_dates).reshape(-1, 1)
seoul_germ_array = np.hstack((seoul_germ_array, np_ffd))

df_seoul_germ = pd.DataFrame(seoul_germ_array[:,(0,1,2,5)], columns=['stnId', 'year', 'budding date', 'flowering date'], index=None)
df_seoul_germ.to_csv(path + 'seoul_data.csv')

df_seoul_flow = pd.DataFrame(seoul_flow_array[:,(0,2,4)], columns=['year', 'flowering date', 'critical date'], index=None)
df_seoul_flow.to_csv(path + 'flowering_data.csv')

## Chill Days Model

In [ ]:
seoul_chill = []
for i, year in enumerate(seoul_germ_array[:,1]):
    critical_temp = 3.34
    budding_day = get_day('g', 108, year)
    chill_days = calculate_chill_day(year, 108, budding_day, critical_temp)
    if chill_days > 0:
        seoul_chill.append(chill_days)
    else:
        seoul_chill.append(np.nan)

avg_chill = 0
count = 0
for i in seoul_chill:
    if i > 0:
        count += 1
        avg_chill += i
avg_chill = avg_chill / count

################################################################################

seoul_chill_gdd = []
for i, year in enumerate(seoul_germ_array[:,1]):
    critical_temp = 3.34
    budding_day = get_day('g', 108, year)
    chill_days_gdd = calculate_chill_gdd(year, 108, budding_day, critical_temp)
    if chill_days_gdd >= 0 or chill_days_gdd < 0:
        seoul_chill_gdd.append(chill_days_gdd)
    else:
        seoul_chill_gdd.append(np.nan)

print(seoul_chill_gdd)


avg_chill_gdd = 0
count = 0
for i in seoul_chill_gdd:
    if i >= 0 or i < 0:
        count += 1
        avg_chill_gdd += i
avg_chill_gdd = avg_chill_gdd / count
print('average GDD : {}'.format(avg_chill_gdd))

In [ ]:
critical_temp = 3.34
years = []
chill_critical_dates_gdd = []

for i, year in enumerate(seoul_germ_array[:,1]):
    for j in range(200):
        chill_days_gdd = calculate_chill_gdd(year, 108, j, critical_temp)
        if chill_days_gdd <= -450:
            chill_critical_dates_gdd.append(j)
            break
        elif j == 199:
            chill_critical_dates_gdd.append(np.nan)
            break

avg = [avg_chill_gdd] * len(seoul_germ_array[:,1])

In [ ]:
# critical date 에 대해
seoul_gdd_chill = []
for i, crit in enumerate(chill_critical_dates_gdd):    
    year = seoul_germ_array[i, 1]
    gdd = 0
    for j, y in enumerate(data_dict['108'][:,0]):
        if y == year:
            if data_dict['108'][j,3] >= crit and data_dict['108'][j,3] < seoul_germ_array[i, 5]:
                gdd += max(0, data_dict['108'][j,4] - 1)
    seoul_gdd_chill.append(gdd)

print(seoul_gdd_chill)

avg_gdd = 0
count = 0
for i in seoul_gdd_chill:
    if i > 0:
        count += 1
    avg_gdd += i
avg_gdd = avg_gdd / count

# budding - flowering을 전부 chill daydates 합으로 계산한 경우 (seoul_flow_crit)
seoul_flow_crit_chill = []
for i, year in enumerate(seoul_germ_array[:,1]):
    gdd = 0
    crit = chill_critical_dates_gdd[i]
    for j, y in enumerate(data_dict['108'][:,0]):
        if y == year:
            if data_dict['108'][j,3] >= crit:
                gdd += max(0, data_dict['108'][j,4] - 1)
            if gdd >= avg_gdd:
                seoul_flow_crit_chill.append(data_dict['108'][j,3])
                break

In [ ]:
rmse = 0
count = 0
for i, crit in enumerate(seoul_flow_crit_chill):
    if crit > 0:
        if seoul_germ_array[i+1,5] > 0:
            rmse += (crit - seoul_germ_array[i+1,5]) ** 2
            count += 1
rmse = np.sqrt(rmse / count)
print('발아를 거쳤을 때 개화 시기 RMSE : {}'.format(rmse))

## GDD Model

In [ ]:
# critical date 에 대해
seoul_gdd = []
for i, crit in enumerate(seoul_germ_array[:,4]):    
    year = seoul_germ_array[i,1]
    gdd = 0
    for j, y in enumerate(data_dict['108'][:,0]):
        if y == year:
            if data_dict['108'][j,3] >= crit and data_dict['108'][j,3] < seoul_germ_array[i, 5]:
                gdd += data_dict['108'][j,4]
    seoul_gdd.append(gdd)

avg_gdd = 0
count = 0
for i in seoul_gdd:
    if i > 0:
        count += 1
    avg_gdd += i
avg_gdd = avg_gdd / count

In [ ]:
# budding - flowering을 전부 heat daydates 합으로 계산한 경우 (seoul_flow_crit)
seoul_flow_crit = []
for i, year in enumerate(seoul_germ_array[:,1]):
    gdd = 0
    crit = seoul_germ_array[i,4]
    for j, y in enumerate(data_dict['108'][:,0]):
        if y == year:
            if data_dict['108'][j,3] >= crit:
                gdd += max(0, data_dict['108'][j,4] - 1)
            if gdd >= avg_gdd:
                seoul_flow_crit.append(data_dict['108'][j,3])
                break

In [ ]:
rmse = 0
count = 0
for i, crit in enumerate(seoul_flow_crit):
    if crit > 0:
        if seoul_germ_array[i, 5] > 0:
            rmse += (crit - seoul_germ_array[i, 5]) ** 2
            count += 1
rmse = np.sqrt(rmse / count)
print('발아를 거쳤을 때 개화 시기 RMSE : {}'.format(rmse))

rmse = 0
count = 0
for i, crit in enumerate(seoul_flow_array[:,4]):
    if crit > 0:
        if seoul_germ_array[i, 5] > 0:
            rmse += (crit - seoul_flow_array[i,2]) ** 2
            count += 1
rmse = np.sqrt(rmse / count)
print('발아를 거치지 않았을 때 개화 시기 RMSE : {}'.format(rmse))

## Runtime 끊김 방지 코드

In [ ]:
i = 1
while i > 0:
    i += 1
    i -= 1